In [ ]:
import json, re, requests, csv, datetime, time
dt=datetime.timedelta(hours=1)

with open("./credentials.json") as f:
    df=json.load(f)
API_Key=df["API_Key"]
API_Secret_Key=df["API_Secret_Key"]
Bearer_Token=df["Bearer_Token"]
if not (API_Key and API_Secret_Key and Bearer_Token):
    print("not enough credentials")

In [ ]:
def create_url_for_answer(start_date,end_date):
    base_url="https://api.twitter.com/2/tweets/search/all"
        # allですべてのTweetに、recentで直近7日間にアクセスできる(all is academic only)
    
    options=[]
        # 表示内容(queryも含む)
    query=[]
        # 検索内容(必須)
    query.append("な")
    query.append('-"ハズレ"')
    query.append('-"おは"')
    query.append('-"よろ"')
    query.append('-"こん"')
    query.append('-"あり"')
    query.append('-"はじめまして"')
    query.append('-"企画"')
    query.append('-"くん"')
    query.append('-"ちゃん"')
    query.append('-"さん"')
    query.append("is:reply")
    query.append("lang:ja")
    query.append("-has:media")
    query.append("-has:links")
    query.append("-has:hashtags")
    query.append("-is:quote")
    query.append("-is:retweet")
    query.append("-is:nullcast") #except for twitter_for_Ads
    query.append("-is:verified")

    options.append("query=" + " ".join(query))
    options.append("start_time={}Z".format(start_date.isoformat()))
    options.append("end_time={}Z".format(end_date.isoformat()))
    options.append("max_results=100")    # returnするツイート数(10~100)
    options.append("tweet.fields=referenced_tweets,source")
    options.append("expansions=author_id")  # 返信元のツイートID
    
    query_param="&".join(options)
    url = base_url+"?"+query_param
#     print(url)
    return url

def create_url_for_question():
    base_url="https://api.twitter.com/2/tweets"
    options=[]
        # 表示内容(queryも含む)

    options.append("ids=" + ",".join(question_ids))
    options.append("tweet.fields=referenced_tweets")
    options.append("expansions=in_reply_to_user_id,author_id")  # 返信先のツイートID
#     options.append("expansions=author_id")  # 返信元のツイートID
    
    query_param="&".join(options)
    url = base_url+"?"+query_param
#     print(url)
    return url

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

def search(url):
    headers = create_headers(Bearer_Token)
    json_response = connect_to_endpoint(url, headers)
    return json_response

def generate_question_ids(answers):
    question_ids=[]
    for i in range(len(answers)):
        answer=answers[i]
        pop_list=[]
        if "referenced_tweets" not in answer or len(answer["referenced_tweets"])>1 or answer["referenced_tweets"][0]["type"]!="replied_to":
            pop_list.append(i)
        else:
            question_id=answer["referenced_tweets"][0]["id"]
            question_ids.append(question_id)
    for i in pop_list:
        answers.pop(i)
    return question_ids
    # print(question_ids)

def delete_mention(text):
    text=re.sub(r"@[a-zA-Z_0-9]+\s+","",text) #メンション削除
    return text

def write_to_file(filename, questions, answers):
    for question, answer in zip(questions,answers):
        q_text,a_text=delete_mention(question["text"]), delete_mention(answer["text"])
        if not (q_text and a_text):
            continue
        with open(filename,"a") as f:
            writer=csv.writer(f)
            writer.writerow([q_text,a_text])

def search_period(file_name):
    #2021-05-22～2021-05-23開始
    with open(file_name,"r") as f:
        until=f.read()
        until=datetime.datetime.strptime(until,"%Y-%m-%d %H:%M:%S")
        since=until-dt
    with open(file_name,"w") as f:
        f.write(since.strftime("%Y-%m-%d %H:%M:%S"))
    return since, until

In [ ]:
SEARCH_PERIODS=100
day_log_path="./now.txt"
QandA_any_path="./tweets/raw_any_QandA.csv"
QandA_1to2_path="./tweets/raw_1to2_QandA.csv"
QandA_2to3_path="./tweets/raw_2to3_QandA.csv"

# init
with open(day_log_path,"w") as f:
    f.write("2021-05-23 00:00:00")
#     f.write("2021-05-21")
with open(QandA_any_path,"w"):
    pass
with open(QandA_1to2_path,"w"):
    pass
with open(QandA_2to3_path,"w"):
    pass

for itr in range(SEARCH_PERIODS):
    since,until = search_period("./now.txt")
    print("it's {}/{} search for {} to {}".format(itr+1, SEARCH_PERIODS, since, until), end="\r")
    raw_answers=search(create_url_for_answer(since,until))
    # print(json.dumps(raw_answers, indent=4, sort_keys=True))
    answers=raw_answers["data"]
    question_ids = generate_question_ids(answers)

    raw_questions=search(create_url_for_question())
    # print(json.dumps(raw_questions, indent=4, sort_keys=True))
    questions=raw_questions["data"]

    raw_questions, raw_answers=[],[]
    raw_1to2_questions, raw_1to2_answers=[],[]
    raw_2to3_questions, raw_2to3_answers=[],[]

    for i in range(len(questions)):
        question, answer = questions[i], answers[i]
        if question["author_id"]==answer["author_id"]:
            continue
        # A->B or A->[B->A]
        raw_questions.append(question)
        raw_answers.append(answer)
        if "in_reply_to_user_id" not in question:
            # A->B
            raw_1to2_questions.append(question)
            raw_1to2_answers.append(answer)
        elif  question["in_reply_to_user_id"]==answer["author_id"]:
            # A->[B->A]
            raw_2to3_questions.append(question)
            raw_2to3_answers.append(answer)

    # 自分自身へのリプライを除く、すべての対話セットを記録する
    write_to_file(QandA_any_path, raw_questions, raw_answers)

    # 対話セットのうち1to2以降のものについて記録する
    write_to_file(QandA_1to2_path,raw_1to2_questions, raw_1to2_answers)

    # 対話セットのうち2to3以降のものについて記録する
    write_to_file(QandA_2to3_path,raw_2to3_questions, raw_2to3_answers)
    time.sleep(3) # 15min/300request = 3sec/request
print("done!")